## BENDERS DECOMPOSITION METHOD
### Prepared by Bikey SERANILLA
#### Large-scale LP

$$\begin{align}\min\ & 150x_1 + 230x_2 + 260x_3 \\
& -\frac{1}{3}(170w_{11} - 238y_{11} + 150w_{21} - 210y_{21} + 36w_{31} + 10w_{41}) \\
& -\frac{1}{3}(170w_{12} - 238y_{12} + 150w_{22} - 210y_{22} + 36w_{32} + 10w_{42}) \\
& -\frac{1}{3}(170w_{13} - 238y_{13} + 150w_{23} - 210y_{23} + 36w_{33} + 10w_{43}) \\
s.t. \\
& x_1 + 2x_2 + x3 \leq 500 \\
& 3x_1 + y_{11} - w_{11} \geq 200 \\
& 3.6x_2 + y_{21} - w_{21} \geq 240 \\
& w_{31} + w_{41} \leq 24x_3 \\
& w_{31} \leq 6000 \\
& 2.5x_1 + y_{12} - w_{12} \geq 200 \\
& 3x_2 + y_{22} - w_{22} \geq 240 \\
& w_{32} + w_{42} \leq 20x_3 \\
& w_{32} \leq 6000 \\
& 2x_1 + y_{13} - w_{13} \geq 200 \\
& 2.4x_2 + y_{23} - w_{23} \geq 240 \\
& w_{33} + w_{43} \leq 16x_3 \\
& w_{33} \leq 6000 \\
& x, y, w \geq 0, y \in \mathbb{Z}\\
\end{align}$$ 


#### Master Model

$$\begin{align}\min\ & 150x_1 + 230x_2 + 260x_3 + V \\
\end{align}$$

$$\begin{align}s.t. \\
& x_1 + 2x_2 + x3 \leq 500 \\
& V \geq 0 \\
&  V \geq (b-F^y)^Tu \\
& V \geq 0, y \in \mathbb{Z}\\
\end{align}$$ 

#### Subroblem Model

$$\begin{align}\min\ & - 170w_{11} + 238y_{11} - 150w_{21} + 210y_{21} - 36w_{31} - 10w_{41}\\
\end{align}$$
$$\begin{align}s.t. \\
& \xi_1 x_1 + y_{11} - w_{11} \geq 200 \\
& \xi_2 x_2 + y_{21} - w_{21} \geq 240 \\
& w_{31} + w_{41} \leq \xi_3 x_3 \\
& w_{31} \leq 6000 \\
& x_1, x_2 \geq 0\\
\end{align}$$ 

$$\begin{align}
\xi = [[3, 3.6, 24],
[2.5, 3, 20],
[2, 2.4, 16]]\\
\end{align}$$

In [ ]:
from IPython.display import Image
Image(filename = "L-Shaped Algorithm.png", width = 400, height = 100)

In [93]:
import gurobipy as gp
from gurobipy import GRB
from numpy.random import randint, binomial
import numpy as np
from math import sqrt
from seaborn import distplot
from math import inf, isclose
import math as math

In [94]:
# Parameters
MAX_AREA = 500
T=2

CROPS = 3
PLANTING_COST = np.array([150, 230, 260])
MIN_QUANTITIES = np.array([200.0, 240.0, 0.0])
QUOTA_MAX = np.array([1000000, 1000000, 6000])
SELL_IN_QUOTA = np.array([170.0, 150.0, 36.0])
SELL_NO_QUOTA = np.array([0.0, 0.0, 10.0])
BUY_PRICE = -np.array([238.0, 210.0, 1000.0])
MEAN_YIELD = np.array([2.5, 3.0, 20.0])
YIELD_MULTIPLIER = {'good':1.2, 'fair':1.0,'bad':0.8 }
YIELD = np.array([[2.5*1.2, 3*1.2, 20*1.2],
                  [2.5*1, 3*1.0, 20*1],
                  [2.5*0.8, 3*0.8, 20*0.8]])

In [95]:
# MASTER PROBLEMS (MP)
# Master Problem 1 (initial MP)
masterProblem_1 = gp.Model()
masterProblem_1.Params.LogToConsole = 0
masterProblem_1.ModelSense = GRB.MINIMIZE
x = masterProblem_1.addVars(CROPS, vtype=GRB.CONTINUOUS, name='x')
masterProblem_1.addConstr(gp.quicksum(x[i] for i in range(CROPS)) <= MAX_AREA)
masterProblem_1.setObjective(gp.quicksum(PLANTING_COST[i]*x[i] for i in range(CROPS)), GRB.MINIMIZE)

# Master Problem 2 (MP with cuts)
masterProblem_2 = gp.Model()
masterProblem_2.Params.LogToConsole = 0
masterProblem_2.ModelSense = GRB.MINIMIZE
x = masterProblem_2.addVars(CROPS, vtype=GRB.CONTINUOUS, name='x')
V = masterProblem_2.addVar(name='V', lb=-99999999)
masterProblem_2.addConstr(gp.quicksum(x[i] for i in range(CROPS)) <= MAX_AREA)
masterProblem_2.setObjective(gp.quicksum(PLANTING_COST[i]*x[i] for i in range(CROPS)) + V, GRB.MINIMIZE)

In [96]:
# SUBPROBLEM
def subProblem(xi, x):
    subProblem = gp.Model()
    subProblem.Params.LogToConsole = 0
    subProblem.ModelSense = GRB.MINIMIZE
    y = subProblem.addVars(CROPS, vtype=GRB.CONTINUOUS, name='y')
    w = subProblem.addVars(4, vtype=GRB.CONTINUOUS, name='w')
    subProblem.addConstr(xi[0]*x[0] + y[0] - w[0] >= MIN_QUANTITIES[0], name="constr1")
    subProblem.addConstr(xi[1]*x[1] + y[1] - w[1] >= MIN_QUANTITIES[1], name="constr2")
    subProblem.addConstr(w[2] + w[3] <= xi[2]*x[2], name="constr3")
    subProblem.addConstr(w[2] <= 6000, name="constr4")
    subProblem.setObjective(238*y[0] - 170*w[0] + 210*y[1] - 150*w[1] - 36*w[2] - 10*w[3], GRB.MINIMIZE)
    subProblem.optimize()
    y_star = subProblem.getAttr('X', y)
    w_star = subProblem.getAttr('X', w)
    u_star = subProblem.Pi
    ObjVal = subProblem.ObjVal

    return y_star, w_star, u_star, ObjVal
    

In [97]:
# L-SHAPED DECOMPOSITION ALGORITHM

J = 10
UB = math.inf
LB = 0
w_0 = 0
w = 0
count = 0

#Scenarios
S = 3
SCENARIO = np.array([[3, 3.6, 24],
                     [2.5, 3, 20],
                     [2, 2.4, 16]])
h = np.array([200, 240, 0, 6000])
T = np.array([[[3, 0, 0, 0],
               [0, 3.6, 0, 0],
               [0, 0, 24, 0]],
              [[2.5, 0, 0, 0],
               [0, 3, 0, 0],
               [0, 0, 20, 0]],
              [[2, 0, 0, 0],
               [0, 2.4, 0, 0],
               [0, 0, 16, 0]]])

#Second-stage Values
ssp_y = np.zeros((S,CROPS)) #y_variables
ssp_z = np.zeros((S,4))     #z_variables
ssp_u = np.zeros((S,4))     #duals / constraints
ssp_o = np.zeros((S))       #objective value

for j in range(J):
# while(w != w_0):

#     print("Iteration Number: ", count)

    # if count == 0:
    if j == 0:
        #Step 1 : Solve first-stage problem
        FSP = masterProblem_1.optimize()
        FSP_ObjVal = masterProblem_1.ObjVal
        x_j = masterProblem_1.getAttr('X', x)
        print(x_j)
        print(FSP_ObjVal)
    else:
        #Step 1.2 : Add cut variable (V) and update objective function
        FSP = masterProblem_2.optimize()
        FSP_ObjVal = masterProblem_2.ObjVal
        x_j = masterProblem_2.getAttr('X', x)
        print(x_j)
        print(FSP_ObjVal)
    
    #Step 2: Solve each second-stage problem
    for s in range(S):
        #ssp = [y_star, w_star, u_star, ObjVal]
        ssp = subProblem(SCENARIO[s], x_j)
        ssp_y[s] = ssp[0].values()
        ssp_z[s] = ssp[1].values()
        ssp_u[s] = np.array(ssp[2])
        ssp_o[s] = ssp[3]
    # print(ssp_u[0])
    # print(T[0].T)
    # print(ssp_z)
    
    #Step 3: Generate optimality cuts
    #Multi-cut approach
    # for s in range(S):
    #     e = np.dot(ssp_u[s].T,h)        # RHS *dot* duals
    #     E = np.dot(ssp_u[s].T,T[s].T)   # Technology Matrix *dot* Duals
    #     masterProblem_2.addConstr(V >= e - gp.quicksum(E[i]*x[i] for i in range(CROPS)))

    #Single-cut approach
    e = sum(np.dot(ssp_u[i].T,h) for i in range(S))/S
    E = sum(np.dot(ssp_u[i].T,T[i].T) for i in range(S))/S
    masterProblem_2.addConstr(V >= e - gp.quicksum(x[i]*E[i] for i in range(CROPS)))

    #Step 4: Stopping criterion
    w_0 = w
    w = e - sum(E[i]*x_j[i] for i in range(S))
    print("w =", w)

    # masterProblem_2.addConstr(V >= w)

    # Update bounds
    LB = max(LB, FSP_ObjVal)
    UB = min(UB, LB + np.mean(ssp_o))
    bound = UB - LB

    # count = count + 1
    j = j + 1
print("LB =", LB)

{0: 0.0, 1: 0.0, 2: 0.0}
0.0
w = 98000.0
{0: 500.0, 1: 0.0, 2: 0.0}
-124500.0
w = -128100.0
{0: 80.0, 1: 420.0, 2: 0.0}
-105600.0
w = -152093.3333333333
{0: 300.2364066193854, 1: 199.76359338061462, 2: 0.0}
-78070.44917257682
w = -147494.08983451535
{0: 420.0, 1: 80.0, 2: 0.0}
-63100.0
w = -143540.0
{0: 411.1111111111111, 1: 88.88888888888889, 2: 0.0}
-62611.11111111111
w = -144188.88888888888
{0: 420.0, 1: 80.0, 2: 0.0}
-62140.0
w = -143540.0
{0: 420.0, 1: 80.0, 2: 0.0}
-62140.0
w = -143540.0
{0: 420.0, 1: 80.0, 2: 0.0}
-62140.0
w = -143540.0
{0: 420.0, 1: 80.0, 2: 0.0}
-62140.0
w = -143540.0
LB = 0
